In [1]:
!pip install transformers datasets accelerate -q  # Install necessary libraries (if not already installed)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is

In [2]:
from transformers import BigBirdTokenizerFast, BigBirdForQuestionAnswering

tokenizer = BigBirdTokenizerFast.from_pretrained(
    "google/bigbird-roberta-large"
)

model = BigBirdForQuestionAnswering.from_pretrained(
    "google/bigbird-roberta-large"
)

# Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# BigBird-large has exactly 24 encoder layers
num_layers = len(model.bert.encoder.layer)
N = 4  # Unfreeze last 4 layers

# Unfreeze the last N encoder layers
for i in range(num_layers - N, num_layers):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = True

# Unfreeze QA classifier head (start/end logits)
for param in model.qa_classifier.parameters():
    param.requires_grad = True

# Print summary
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable parameters: {trainable}/{total}")

2025-11-11 02:54:12.839732: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762829653.223024      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762829653.415778      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-large and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: 58782722/366472194


In [3]:
max_len = 2048
doc_stride = 128

In [4]:
import json, os
from datasets import Dataset

def load_webmd_squad_v2_json(path):
    with open(path, "r", encoding="utf-8") as f:
        raw = json.load(f)

    rows = []
    for article in raw.get("data", []):
        for para in article.get("paragraphs", []):
            context = para.get("context", "")
            for qa in para.get("qas", []):
                qid = qa.get("id")
                question = qa.get("question", "")
                is_impossible = qa.get("is_impossible", False)

                # answers is a LIST of dicts; normalize to SQuAD schema:
                texts, starts = [], []
                for a in qa.get("answers", []):
                    if isinstance(a, dict):
                        t = a.get("text")
                        s = a.get("answer_start", None)

                        # Some variants include 'answer_starts': try to salvage if 'answer_start' is None
                        if s is None:
                            s_list = a.get("answer_starts")
                            if isinstance(s_list, list) and s_list:
                                # pick first int if present (can be nested)
                                s = s_list[0][0] if isinstance(s_list[0], list) else s_list[0]

                        if t is not None and isinstance(s, int):
                            texts.append(str(t))
                            starts.append(int(s))

                # For SQuAD v2, unanswerable = empty lists
                if is_impossible or len(texts) == 0:
                    texts, starts = [], []

                rows.append({
                    "id": qid,
                    "question": question,
                    "context": context,
                    "answers": {"text": texts, "answer_start": starts},
                    "is_impossible": bool(is_impossible),
                })

    return Dataset.from_list(rows)



In [5]:
train_ds = load_webmd_squad_v2_json("/kaggle/input/mashqa/mashqa_data/train_webmd_squad_v2_full.json")  # <-- your train file path here
val_ds   = load_webmd_squad_v2_json("/kaggle/input/mashqa/mashqa_data/val_webmd_squad_v2_full.json")  # <-- your val/test file path here

print(train_ds)
print(val_ds[:2]["question"])

Dataset({
    features: ['id', 'question', 'context', 'answers', 'is_impossible'],
    num_rows: 27728
})
['What are the symptoms of heart failure?', 'Can high blood pressure bring on heart failure?']


In [6]:
def prepare_train_features(examples):

    # Tokenize question-context pairs
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_len,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):

        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        example_idx = sample_mapping[i]

        # -----------------------
        # (1) Extract answer dict
        # -----------------------
        answer_list = examples["answers"][example_idx]

        # If dataset provides list of strings directly → wrap it
        if isinstance(answer_list, dict):
            answer_list = [answer_list]

        # -----------------------
        # (2) Find first valid answer
        # -----------------------
        start_char = None
        end_char = None

        for ans in answer_list:
            s = ans.get("answer_start", None)
            e = ans.get("answer_end", None)
            txt = ans.get("text", "")

            if s is None or e is None:
                continue
            if isinstance(s, list):
                s = s[0] if len(s) else None
            if isinstance(e, list):
                e = e[0] if len(e) else None

            if s is not None and e is not None:
                start_char = s
                end_char = e
                break

        # Case: no valid answer → label CLS
        if start_char is None or end_char is None:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        # -----------------------
        # (3) Sequence ID mapping
        # -----------------------
        sequence_ids = tokenized.sequence_ids(i)

        # Locate context token span
        context_start = None
        context_end = None
        for j, sid in enumerate(sequence_ids):
            if sid == 1 and context_start is None:
                context_start = j
            if sid == 1:
                context_end = j

        if context_start is None:
            # No context at all → CLS
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        # -----------------------
        # (4) Answer outside context?
        # -----------------------
        if not (offsets[context_start][0] <= start_char <= offsets[context_end][1]):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        if not (offsets[context_start][0] <= end_char <= offsets[context_end][1]):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
            continue

        # -----------------------
        # (5) Convert char → token
        # -----------------------
        start_token = context_start
        while start_token <= context_end and offsets[start_token][0] <= start_char:
            start_token += 1
        start_token -= 1

        end_token = context_end
        while end_token >= context_start and offsets[end_token][1] >= end_char:
            end_token -= 1
        end_token += 1

        start_positions.append(start_token)
        end_positions.append(end_token)

    # Attach to return dict
    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions

    return tokenized


In [7]:
import numpy as np
import collections

def compute_metrics(p):
    def normalize(s):
        import re, string
        s = s.lower().strip()
        s = ''.join(ch for ch in s if ch not in string.punctuation)
        s = re.sub(r"\b(a|an|the)\b", " ", s)
        return " ".join(s.split())

    def f1(pred, truth):
        pred_t = normalize(pred).split()
        true_t = normalize(truth).split()
        common = collections.Counter(pred_t) & collections.Counter(true_t)
        if len(common) == 0: return 0
        precision = sum(common.values()) / len(pred_t)
        recall = sum(common.values()) / len(true_t)
        return 2 * precision * recall / (precision + recall + 1e-8)

    start_preds = np.argmax(p.predictions[0], axis=-1)
    end_preds   = np.argmax(p.predictions[1], axis=-1)

    true_starts = p.label_ids[:,0]
    true_ends   = p.label_ids[:,1]

    em = 0
    f1_total = 0

    for i in range(len(start_preds)):
        pred_ans = tokenizer.decode(
            p.predictions[0][i][start_preds[i]:end_preds[i]+1]
        )
        true_ans = tokenizer.decode(
            p.label_ids[i][true_starts[i]:true_ends[i]+1]
        )

        if normalize(pred_ans) == normalize(true_ans):
            em += 1

        f1_total += f1(pred_ans, true_ans)

    return {
        "exact_match": em / len(start_preds),
        "f1": f1_total / len(start_preds)
    }


In [8]:

# Map (note: batched=True and we REMOVE old cols to avoid clutter)
tokenized_train = train_ds.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_ds.column_names
)
tokenized_val = val_ds.map(
    prepare_train_features,
    batched=True,
    remove_columns=val_ds.column_names
)


train_dataset = tokenized_train
val_dataset   = tokenized_val


Map:   0%|          | 0/27728 [00:00<?, ? examples/s]

Map:   0%|          | 0/3587 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="bigbird-checkpoints",

    # ✅ STEP-BASED evaluation & saving
    eval_strategy="steps",
    eval_steps=1000,                  # evaluate every 1000 steps
    save_strategy="steps",
    save_steps=2000,                  # save checkpoint every 2000 steps

    learning_rate=3e-5,
    weight_decay=0.01,

    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,

    gradient_accumulation_steps=2,    # effective batch size = 4

    num_train_epochs=1,               # keep training ≈ 7–8 hours

    warmup_steps=200,

    fp16=False,
    bf16=True,                        # safe for GPU/TPU

    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",

    report_to="none",
)


In [10]:
# ✅ IMPORTANT FIX: Trainer expects labels as dict keys inside batches:
#   {"start_positions": ..., "end_positions": ...}
#   So we must ensure features returned from dataset map contain those keys.

def collate_fn(features):
    # Trainer automatically handles this but custom code avoids Rare BigBird issues.
    batch = {
        "input_ids": np.array([f["input_ids"] for f in features]),
        "attention_mask": np.array([f["attention_mask"] for f in features]),
        "start_positions": np.array([f["start_positions"] for f in features]),
        "end_positions": np.array([f["end_positions"] for f in features]),
    }
    return batch


In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


In [12]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,   # ✅ use HF QA collator
    tokenizer=tokenizer
)


/tmp/ipykernel_19/412871800.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
1000,0.065200,0.020972
2000,0.042900,0.021077
3000,0.050000,0.018143


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=3801, training_loss=0.11512009906113022, metrics={'train_runtime': 22084.9779, 'train_samples_per_second': 1.377, 'train_steps_per_second': 0.172, 'total_flos': 1.16089963997184e+17, 'train_loss': 0.11512009906113022, 'epoch': 1.0})

In [14]:
trainer.save_model("./bigbird-roberta-large-mashqa")

In [15]:
!rm -rf /kaggle/working/bigbird-checkpoints

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
!zip -r bigbird_finetuned_model.zip /kaggle/working/bigbird-roberta-large-mashqa

  adding: kaggle/working/bigbird-roberta-large-mashqa/ (stored 0%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/special_tokens_map.json (deflated 83%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/training_args.bin (deflated 51%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/spiece.model (deflated 69%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/tokenizer.json (deflated 90%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/tokenizer_config.json (deflated 94%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/config.json (deflated 52%)
  adding: kaggle/working/bigbird-roberta-large-mashqa/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
